Project: Sentiment classifer for Amazon reviews
Dataset: Amazon Polarity - https://huggingface.co/datasets/fancyzhx/amazon_polarity
Model: distilbert-base-uncased

In [1]:
%pip install torch numpy peft datasets transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
from peft import LoraConfig, get_peft_model
from transformers import Trainer, AutoTokenizer
import numpy as np


In [4]:
# get test and train data and tokenize 

dataset = load_dataset("amazon_polarity")

train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def process_text(example):
    return tokenizer(example["content"], truncation=True, padding="max_length")

tokenized_train_dataset = train_dataset.map(process_text, batched=True)
tokenized_test_dataset = test_dataset.map(process_text, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 5541.70 examples/s]


In [5]:
# load pretrained model "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', 
id2label={0: "NEGATIVE", 1: "POSITIVE"},
label2id={"NEGATIVE": 0, "POSITIVE": 1},
num_labels=2)

# args for training 
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",
    num_train_epochs=5,
    do_eval=True,
    save_strategy='epoch',
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions == labels).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_train_dataset

)

initial_evaluation = trainer.evaluate()
print("Initial Evaluation:", initial_evaluation) 


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Source\GenerativeAINanodegree-project1\.venv\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 250/250 [02:41<00:00,  1.55it/s]

Initial Evaluation: {'eval_loss': 0.6944077014923096, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.489, 'eval_runtime': 162.0178, 'eval_samples_per_second': 6.172, 'eval_steps_per_second': 1.543}


In [6]:
# turn model into peft model with lora, targeting classifer layer for training
config = LoraConfig(target_modules=["classifier"],)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 6,160 || all params: 66,961,170 || trainable%: 0.0092


In [7]:
# create trainer and save trained model 

ft_training_args = TrainingArguments(
    output_dir="./lora_results",
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    num_train_epochs=5,
    do_eval=True,
    save_strategy='epoch',
)

ft_trainer = Trainer(
    model=lora_model,
    args=ft_training_args,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_train_dataset

)

ft_trainer.train()

lora_model.save_pretrained("./fine-tuned")

                                                 
 20%|██        | 125/625 [07:44<19:55,  2.39s/it]

{'eval_runtime': 166.6899, 'eval_samples_per_second': 5.999, 'eval_steps_per_second': 0.75, 'epoch': 1.0}


                                                   
 40%|████      | 250/625 [20:07<14:32,  2.33s/it]

{'eval_runtime': 445.4813, 'eval_samples_per_second': 2.245, 'eval_steps_per_second': 0.281, 'epoch': 2.0}


                                                     
 60%|██████    | 375/625 [28:00<09:56,  2.39s/it]

...

{'eval_runtime': 170.0917, 'eval_samples_per_second': 5.879, 'eval_steps_per_second': 0.735, 'epoch': 4.0}


                                                   
100%|██████████| 625/625 [43:38<00:00,  2.38s/it]

{'eval_runtime': 166.9277, 'eval_samples_per_second': 5.991, 'eval_steps_per_second': 0.749, 'epoch': 5.0}


100%|██████████| 625/625 [43:38<00:00,  4.19s/it]


{'train_runtime': 2618.5554, 'train_samples_per_second': 1.909, 'train_steps_per_second': 0.239, 'train_loss': 0.6942584228515625, 'epoch': 5.0}


In [8]:
#evaluate fine tuned/trained model 

final_model = AutoModelForSequenceClassification.from_pretrained("./fine-tuned")

final_trainer = Trainer(
	args=ft_training_args,
	compute_metrics=compute_metrics,
	model=final_model,
	eval_dataset=tokenized_test_dataset,
	train_dataset=tokenized_train_dataset,
)

final_evaluation = final_trainer.evaluate()
print("final evaluation:", final_evaluation)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 125/125 [02:47<00:00,  1.34s/it]

final evaluation: {'eval_loss': 0.6912226676940918, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.539, 'eval_runtime': 168.7991, 'eval_samples_per_second': 5.924, 'eval_steps_per_second': 0.741}
